# Image Sliders

This page shows an example of using widgets to interact with frame grabs from the camera. Nearly every frame collected from Deployment #3 is available for viewing, each downloaded from the remote data archive in real-time.

#### Setup the environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import pycamhd as camhd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### Get a list of remote CamHD files to process

In [ ]:
dbcamhd = pd.read_json('dbcamhd.json', orient="records", lines=True)
dbcamhd.tail()

In [ ]:
print("Total files: %i" % len(dbcamhd))
print("Total frames: %i" % dbcamhd.frame_count.sum())

In [ ]:
filenames = list(dbcamhd.filename[(dbcamhd.deployment == 2) & (dbcamhd.frame_count > 5000) & (dbcamhd.frame_count < 30000)])
filenames.sort()
filenames[0]

#### Set up plotting function

In [ ]:
def show_image(file_number, frame_number):
    filename = filenames[file_number]
    plt.rc('figure', figsize=(10, 5))
    plt.rcParams.update({'font.size': 8})
    frame = camhd.get_frame(filename, frame_number, 'rgb24')
    fig, ax = plt.subplots();
    im1 = ax.imshow(frame);
    plt.yticks(np.arange(0,1081,270))
    plt.xticks(np.arange(0,1921,480))
    plt.title('%s %s' % (filename[84:], frame_number));

#### Interact using ipywidgets

In [ ]:
from ipywidgets import interact
from ipywidgets import IntSlider
file_slider = IntSlider(min=0, max=len(filenames)-1, step=1, value=0, continuous_update=False)
frame_slider = IntSlider(min=0, max=camhd.get_frame_count(filenames[0]), step=1, value=0, continuous_update=False)

def update_range(*args):
    frame_slider.max = camhd.get_frame_count(filenames[file_slider.value])-1
    if frame_slider.value > frame_slider.max:
        frame_slider.value = frame_slider.max
file_slider.observe(update_range, 'value')

interact(show_image, file_number=file_slider, frame_number=frame_slider);

### References

 - [PyCamHD](https://github.com/tjcrone/pycamhd)
 - [CamHD Raw Data Archive](https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301)
 - [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/)